# Ejercicio de BYTE-PAIR ENCODING
Integrantes:
*   Aguilar Valenzuela Luis Hector
*   Camargo Loaiza Julio Andres
*   Minjares Neriz Victor Manuel

# Token Learning Example

## Importaciones

In [6]:
import re, collections

## Funciones

### get_vocab(filename)
Función que recibe el nombre de el archivo de texto y devuelve un vocabulario de palabras con la frecuencia de cada palabra y un separador en cada palabra.

In [ ]:
def get_vocab(filename):
    # La funcion defaultdict crea un dictionario vacio
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            # La función strip quita los espacios al principio y al final de un string
            # La función split separa las palabras y las devuelve en un array
            words = line.strip().split()
            
            # Recorre cada palabra del arreglo de palabras
            for word in words:
                # Aqui se llena el diccionario. Agregara al diccionario el elemnto : un espacio +
                # la palabra + el simbolo de fin de palabra </w>
                # NOTA : list() aqui puede ser opcional, tal vez
                vocab[' '.join(list(word)) + ' </w>'] += 1 
    return vocab

### get_tokens(vocab)
Funcion que recibe un diccionario de palabras, despues transforma el diccionario de palabras a uno de letras.

In [ ]:
def get_tokens(vocab):

    # Declara un diccionario vacío
    tokens = collections.defaultdict(int)

    # Iteramos por cada palabra y tomando su respectiva frecuencia
    for word, freq in vocab.items():
        # Separa las palabras por letra
        word_tokens = word.split()
        # Llenamos el diccionario tokens con cada letra y su respectiva frecuencia
        for token in word_tokens:
            # Guarda la palabra en el diccionario y le suma la frecuencia de la palabra.
            tokens[token] += freq
    return tokens

### get_stats(vocab)
Función que recibe un vocabulario (un diccionario con la frecuencia de cada palabra) y devuelve un diccionario con la frecuencia de los bigramas (pares de palabras consecutivos) en el vocabulario.

In [ ]:
def get_stats(vocab):
    # Declara un diccionario vacío
    pairs = collections.defaultdict(int)

    # Iteramos por cada palabra y tomando su respectiva frecuencia
    for word, freq in vocab.items():
        # Separa las palabras por letra
        symbols = word.split()
        # Esto se lee "Recorre el largo de"
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

### merge_vocab(pair, v_in)
Función que recibe una pareja de palabras (pair) y un vocabulario (v_in) y devuelve otro vocabulario nuevo (v_out) con las nuevas parejas de palabras concatenadas.

In [35]:
def merge_vocab(pair, v_in):
    #Crea un diccionario vacío
    v_out = {}
    # Quita los caracteres especiales
    bigram = re.escape(' '.join(pair))
    # Crea expresión regular para buscar un bigrama que esté rodeado por caracteres 
    # no blancos y que no esté precedido ni seguido por otras palabras. 
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    # Reemplaza la cadena bigram con la pareja concatenada y lo agrega en v_out
    for word in v_in:
        # Se remplaza una palabra word por el bigrama 
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    # Regresa el nuevo vocabulario
    return v_out

## Codigo principal

### Corpus a usar

In [65]:
# Minicorpus a usar :
# https://drive.google.com/file/d/17h_rLrWL2xg3jD0U1CCseeaAd6t17yc0/view?usp=share_linket 

vocab = get_vocab('miniCorpus.txt')


Visualización de los tokens antes de aplicarles el algoritmo de Byte Pair Encoding 

In [66]:
# En este bloque de código se muestran un diccionario de todas las palabras en el vocabulario.

print('==========')
print('Tokens Before BPE')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('==========')

Tokens Before BPE
Tokens: defaultdict(<class 'int'>, {'y': 14, 'o': 53, ',': 14, '</w>': 139, 'j': 2, 'u': 18, 'a': 57, 'n': 39, 'g': 7, 'l': 36, 'd': 44, 'e': 91, 'r': 35, 's': 38, 'c': 25, 'i': 42, 'b': 7, 'á': 1, 'm': 13, 't': 23, 'ñ': 3, 'q': 7, 'f': 2, 'h': 9, 'v': 11, 'p': 17, 'é': 1, 'í': 3, ';': 2, '.': 2})
Number of tokens: 30


Algoritmo de Byte Pair Encoding

In [80]:
num_merges = 15
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens = get_tokens(vocab)
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')

Iter: 0
Best pair: ('e', '</w>')
Tokens: defaultdict(<class 'int'>, {'y': 14, 'o': 29, ',': 14, '</w>': 91, 'j': 2, 'u': 18, 'a': 57, 'n': 39, 'g': 7, 'l': 36, 'o</w>': 24, 'd': 44, 'e</w>': 24, 'r': 35, 'e': 67, 's': 38, 'c': 25, 'i': 42, 'b': 7, 'á': 1, 'm': 13, 't': 23, 'ñ': 3, 'q': 7, 'f': 2, 'h': 9, 'v': 11, 'p': 17, 'é': 1, 'í': 3, ';': 2, '.': 2})
Number of tokens: 32
Iter: 1
Best pair: ('a', '</w>')
Tokens: defaultdict(<class 'int'>, {'y': 14, 'o': 29, ',': 14, '</w>': 72, 'j': 2, 'u': 18, 'a': 38, 'n': 39, 'g': 7, 'l': 36, 'o</w>': 24, 'd': 44, 'e</w>': 24, 'r': 35, 'e': 67, 's': 38, 'c': 25, 'i': 42, 'b': 7, 'á': 1, 'm': 13, 'a</w>': 19, 't': 23, 'ñ': 3, 'q': 7, 'f': 2, 'h': 9, 'v': 11, 'p': 17, 'é': 1, 'í': 3, ';': 2, '.': 2})
Number of tokens: 33
Iter: 2
Best pair: ('e', 'n')
Tokens: defaultdict(<class 'int'>, {'y': 14, 'o': 29, ',': 14, '</w>': 72, 'j': 2, 'u': 18, 'a': 38, 'n': 22, 'g': 7, 'l': 36, 'o</w>': 24, 'd': 44, 'e</w>': 24, 'r': 35, 'e': 50, 's': 38, 'c': 25, 'i'

# Encoding and Decoding

## Funciones

### get_vocab(filename)
Función que recibe el nombre de el archivo de texto y devuelve un vocabulario de palabras con la frecuencia de cada palabra y un separador en cada palabra.

In [14]:
def get_vocab(filename):
    # La funcion defaultdict crea un dictionario vacio
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8-sig') as fhand:
        for line in fhand:
            # La función strip quita los espacios al principio y al final de un string
            # La función split separa las palabras y las devuelve en un array
            words = line.strip().split()
            
            # Recorre cada palabra del arreglo de palabras
            for word in words:
                # Aqui se llena el diccionario. Agregara al diccionario el elemnto : un espacio +
                # la palabra + el simbolo de fin de palabra </w>
                # NOTA : Hacemos todo minusculas para reducir el tiempo de ejecucion
                # y el numero de tokens
                # NOTA2 : list() aqui puede ser opcional, tal vez
                vocab[' '.join(list(word.lower())) + ' </w>'] += 1 
    return vocab

### get_stats(vocab)
Función que recibe un vocabulario (un diccionario con la frecuencia de cada palabra) y devuelve un diccionario con la frecuencia de los bigramas (pares de palabras consecutivos) en el vocabulario.

In [3]:
def get_stats(vocab):
    # Declara un diccionario vacío
    pairs = collections.defaultdict(int)

    # Iteramos por cada palabra y tomando su respectiva frecuencia
    for word, freq in vocab.items():
        # Separa las palabras por letra
        symbols = word.split()
        # Esto se lee "Recorre el largo de"
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

### merge_vocab(pair,v_in)
Función que recibe una pareja de palabras (pair) y un vocabulario (v_in) y devuelve otro vocabulario nuevo (v_out) con las nuevas parejas de palabras concatenadas.

In [4]:
def merge_vocab(pair, v_in):
    #Crea un diccionario vacío
    v_out = {}
    # Quita los caracteres especiales
    bigram = re.escape(' '.join(pair))
    # Crea expresión regular para buscar un bigrama que esté rodeado por caracteres 
    # no blancos y que no esté precedido ni seguido por otras palabras. 
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    # Reemplaza la cadena bigram con la pareja concatenada y lo agrega en v_out
    for word in v_in:
        # Se remplaza una palabra word por el bigrama 
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    # Regresa el nuevo vocabulario
    return v_out

### get_tokens_from_vocab(vocab)
Funcion que toma como entrada un diccionario con las frecuencias(vocab) y te regresa dos diccionarios. tokens_frequencies es un diccionario de letras/simbolos con sus frecuencias de aparicion en el vocab. 
vocab_tokenization es un diccionario con las palabras del vocab con su respectiva tokenizacion.

In [7]:
def get_tokens_from_vocab(vocab):
    # Creamos diccionarios vacios
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    # Iteramos por cada palabra y tomando su respectiva frecuencia
    for word, freq in vocab.items():
        # Separamos la palabra en sus letras
        word_tokens = word.split()
        # Llenamos el diccionario con las letras/simbolos y sus frecuencias
        for token in word_tokens:
            tokens_frequencies[token] += freq
        # En este otro lo llenamos con las palabras y su tokenizacion
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(get_vocab('miniCorpus.txt'))

###  measure_token_length(token)
Función que mide la longitud de un token de texto, incluyendo o no un espacio en blanco dependiendo de si el token termina con "\</w>".

In [8]:
def measure_token_length(token):
    # Verificamos si el token es el final de una palabra, esto es
    # si los ultimos 4 caracteres son </w>
    if token[-4:] == '</w>':
        # Si lo es, entonces devolvemos la longitud sin esos 4 
        # caracteres y le agregamos un espacio en blanco
        return len(token[:-4]) + 1
    else:
        # De lo contrario, devolvemos la longitud tal cual
        return len(token)

### tokenize_word(string, sorted_tokens, unknown_token='\</u>')

Funcion que tokeniza una cadena de texto dada, string, en base a una lista de tokens ordenados, sorted_tokens. \</u> es la expresion regular para token desconocido.

In [9]:
def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    #  Si la cadena a tokenizar es un espacio en blanco o una palabra 
    # fuera del vocabulario regresamos una lista vacia o con la palabra
    # desconocida
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token]

    # Arreglo donde guardaremos los tokens
    string_tokens = []
    # Iteramos por cada letra/simbolo del vocabulario
    for i in range(len(sorted_tokens)):
        # Agarramos un elemento del vocabulario
        token = sorted_tokens[i]
        # Evitamos que se tomen caracteres del token como
        # caracteres especiales de expresiones regulares
        token_reg = re.escape(token.replace('.', '[.]'))

        # Se busca todas las coincidencias que tiene el token, token_reg, en la cadena de 
        # texto, string, y se almacenan sus posiciones inicial y final
        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        # Si no hay coincidencias pasamos al siguiente token    
        if len(matched_positions) == 0:
            continue

        # Agarramos las posiciones finales de todas las coincidencias e inicializamos
        # el comienzo en 0, esto es el principio del string a tokenizar
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]
        substring_start_position = 0
        # Iteramos sobre todas las posiciones finales de las coincidencias
        for substring_end_position in substring_end_positions:
            # Dividiremos la cadena en una subcadena
            substring = string[substring_start_position:substring_end_position]
            # Llamamos recursivamente para dividir la subcadena en tokens con una 
            # nueva lista de tokens que contiene todos los tokens en sorted_tokens excepto los 
            # que ya han sido utilizados
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            # Agregamos el token actual a string_tokens
            string_tokens += [token]
            # Actualizamos posicionandonos despues del ultimo token actual
            substring_start_position = substring_end_position + len(token)
        # Por ultimo con la subcadena restante la damos como input en nuestra funcion recursiva
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        # Cuando ya esta todo tokenizado, la funcion
        # devuelve la lista con los tokens
        break
    return string_tokens

## Código principal

### Corpus a utilizar

In [15]:
vocab = get_vocab('donQuijote.txt')
# vocab = get_vocab('miniCorpus.txt')

Se visualizan los tokens antes de aplicarles el algoritmo de Byte Pair Encoding 

In [16]:
print('==========')
print('Tokens Before BPE')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
print('All tokens: {}'.format(tokens_frequencies.keys()))
print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
print('==========')

Tokens Before BPE
All tokens: dict_keys(['e', 'l', '</w>', 'i', 'n', 'g', 'o', 's', 'h', 'd', 'a', 'q', 'u', 'j', 't', 'm', 'c', 'y', ',', 'r', 'b', 'á', 'ñ', 'f', 'v', 'p', 'é', 'í', ';', '.', 'ó', '1', '6', '0', '4', 'ú', 'z', ':', 'x', '¿', '?', '-', '¡', '!', 'ü', '»', "'", '«', '(', ')', '"', 'ï', 'w', ']', 'à', '7', '5', '2', '3', 'ù'])
Number of tokens: 60


Algoritmo de Byte Pair Encoding

In [17]:
num_merges = 2000
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
    print('All tokens: {}'.format(tokens_frequencies.keys()))
    print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
    print('==========')

Iter: 0
Best pair: ('e', '</w>')
All tokens: dict_keys(['e', 'l', '</w>', 'i', 'n', 'g', 'o', 's', 'h', 'd', 'a', 'q', 'u', 'j', 't', 'e</w>', 'm', 'c', 'y', ',', 'r', 'b', 'á', 'ñ', 'f', 'v', 'p', 'é', 'í', ';', '.', 'ó', '1', '6', '0', '4', 'ú', 'z', ':', 'x', '¿', '?', '-', '¡', '!', 'ü', '»', "'", '«', '(', ')', '"', 'ï', 'w', ']', 'à', '7', '5', '2', '3', 'ù'])
Number of tokens: 61
Iter: 1
Best pair: ('a', '</w>')
All tokens: dict_keys(['e', 'l', '</w>', 'i', 'n', 'g', 'o', 's', 'h', 'd', 'a', 'q', 'u', 'j', 't', 'e</w>', 'a</w>', 'm', 'c', 'y', ',', 'r', 'b', 'á', 'ñ', 'f', 'v', 'p', 'é', 'í', ';', '.', 'ó', '1', '6', '0', '4', 'ú', 'z', ':', 'x', '¿', '?', '-', '¡', '!', 'ü', '»', "'", '«', '(', ')', '"', 'ï', 'w', ']', 'à', '7', '5', '2', '3', 'ù'])
Number of tokens: 62
Iter: 2
Best pair: ('o', '</w>')
All tokens: dict_keys(['e', 'l', '</w>', 'i', 'n', 'g', 'o', 's', 'o</w>', 'h', 'd', 'a', 'q', 'u', 'j', 't', 'e</w>', 'a</w>', 'm', 'c', 'y', ',', 'r', 'b', 'á', 'ñ', 'f', 'v', 

KeyboardInterrupt: 

## Probando la tokenización

In [62]:
word_given_known = 'seiscientos</w>'
word_given_unknown = 'parler</w>'

sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

print(sorted_tokens)

['</w>', 'e', 'a', 'o', 's', 'n', 'r', 'l', 'd', 'u', 'i', 't', 'c', 'm', ',', 'p', 'q', 'y', 'b', 'h', 'v', 'g', 'í', 'j', 'ó', '.', 'f', 'é', 'á', '-', 'z', ';', 'ñ', ':', 'ú', '?', '¿', "'", '!', '¡', 'x', '»', '"', 'ü', '(', ')', '«', '1', '6', 'ï', '0', '4', 'w', 'ù', '\ufeff', ']', 'à', '7', '5', '2', '3']


### Con una palabra conocida

In [63]:
word_given = word_given_known 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

Tokenizing word: seiscientos</w>...
Tokenization of the known word:
['s', 'e', 'i', 's', 'c', 'i', 'e', 'n', 't', 'o', 's', '</w>']
Tokenization treating the known word as unknown:
['s', 'e', 'i', 's', 'c', 'i', 'e', 'n', 't', 'o', 's', '</w>']


### Con una palabra desconocida

In [64]:
word_given = word_given_unknown 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

Tokenizing word: parler</w>...
Tokenizating of the unknown word:
['p', 'a', 'r', 'l', 'e', 'r', '</w>']


# Conclusiones



* El costo computacional de la tokenización puede ser bastante alto cuando se trata de textos extensos.
* La limpieza del conjunto de datos es necesaria si queremos que nuestro tokenizador trabaje de la forma que queremos, de lo contrario aprenderá cosas indeseables.
